# Dataset de Microrregiões Brasileiras para Análise

## Extração com mapeamento CORRETO de microrregiões

Este notebook extrai dados da [Base dos Dados](https://basedosdados.org/) usando o mapeamento correto município → microrregião, gerando um dataset consolidado para análises econométricas.


## 0. Como usar este Notebook

### Um único produto:
```python
PRODUTOS_AGRICOLAS = ['Soja']
```

### Múltiplos produtos (agregados):
```python
PRODUTOS_AGRICOLAS = ['Milho', 'Soja']  # Análise conjunta de milho + soja
```

### Exemplos de produtos disponíveis:
- `['Cana-de-açúcar']` - Padrão
- `['Soja']` - Apenas soja
- `['Milho']` - Apenas milho
- `['Café (beneficiado)']` - Café processado
- `['Algodão herbáceo']` - Algodão
- `['Arroz']` - Arroz
- `['Milho', 'Soja', 'Trigo']` - Grãos combinados
- `['Laranja', 'Limão', 'Tangerina']` - Cítricos combinados

O código se adapta automaticamente! Os valores são agregados quando múltiplos produtos são selecionados. 🎯


In [52]:
# Importar bibliotecas
import basedosdados as bd
import pandas as pd
import numpy as np
import os

# Configuração
PROJECT_ID = "bdsdasd"
ANOS = list(range(2003, 2024))

# CONFIGURAÇÃO DOS PRODUTOS AGRÍCOLAS
# Pode ser uma lista com 1 ou mais produtos!
PRODUTOS_AGRICOLAS = ['Cana-de-açúcar']  # Exemplos: ['Soja'], ['Milho', 'Soja'], ['Café (beneficiado)']

# Garantir que sempre seja uma lista
if isinstance(PRODUTOS_AGRICOLAS, str):
    PRODUTOS_AGRICOLAS = [PRODUTOS_AGRICOLAS]

os.makedirs("output", exist_ok=True)

print("✅ Configuração completa!")
print(f"📅 Período: {ANOS[0]}-{ANOS[-1]}")
print(f"🌾 Produtos selecionados: {', '.join(PRODUTOS_AGRICOLAS)}")


✅ Configuração completa!
📅 Período: 2003-2023
🌾 Produtos selecionados: Cana-de-açúcar


## 1. Mapeamento Município → Microrregião

Query correta para obter o mapeamento:


In [53]:
# PASSO 1: Obter mapeamento correto município -> microrregião
query_mapeamento = """
SELECT 
    id_municipio,
    id_microrregiao,
    nome AS nome_municipio,
    sigla_uf
FROM 
    `basedosdados.br_bd_diretorios_brasil.municipio`
"""

print("🗺️ Baixando mapeamento município → microrregião...")
df_municipios = bd.read_sql(query_mapeamento, billing_project_id=PROJECT_ID)

print(f"✅ {len(df_municipios):,} municípios mapeados")
print(f"✅ {df_municipios['id_microrregiao'].nunique()} microrregiões identificadas")
print(f"\nExemplo do mapeamento:")
df_municipios.head()


🗺️ Baixando mapeamento município → microrregião...
Downloading: 100%|██████████|
✅ 5,570 municípios mapeados
✅ 558 microrregiões identificadas

Exemplo do mapeamento:


,id_municipio,id_microrregiao,nome_municipio,sigla_uf
0,1100809,11001,Candeias do Jamari,RO
1,1100338,11001,Nova Mamoré,RO
2,1100205,11001,Porto Velho,RO
3,1101104,11001,Itapuã do Oeste,RO
4,1100940,11001,Cujubim,RO


## 2. Estações Meteorológicas (Tratamento)

Agora vamos agregar as estações usando o mapeamento correto:


In [54]:
# PASSO 2: Estações meteorológicas
query_estacoes = """
SELECT
    e.id_municipio,
    e.id_estacao,
    e.estacao AS nome_estacao,
    EXTRACT(YEAR FROM e.data_fundacao) AS ano_fundacao,
    e.latitude,
    e.longitude
FROM
    `basedosdados.br_inmet_bdmep.estacao` e
WHERE
    e.data_fundacao IS NOT NULL
    AND e.id_municipio IS NOT NULL
"""

print("🌡️ Baixando dados de estações...")
df_estacoes_mun = bd.read_sql(query_estacoes, billing_project_id=PROJECT_ID)

# Fazer o JOIN com o mapeamento
df_estacoes_full = df_estacoes_mun.merge(
    df_municipios[['id_municipio', 'id_microrregiao']], 
    on='id_municipio', 
    how='left'
)

# Agregar por microrregião
df_estacoes = df_estacoes_full.groupby('id_microrregiao').agg({
    'ano_fundacao': 'min',
    'id_estacao': 'count'
}).reset_index()

df_estacoes.columns = ['id_microrregiao', 'primeiro_ano_estacao', 'num_estacoes']

print(f"✅ {len(df_estacoes)} microrregiões com estações ({len(df_estacoes)/558*100:.1f}% de cobertura)")
print(f"✅ Total: {df_estacoes_full['id_estacao'].nunique()} estações")
df_estacoes.head()


🌡️ Baixando dados de estações...
Downloading: 100%|██████████|
✅ 394 microrregiões com estações (70.6% de cobertura)
✅ Total: 610 estações


,id_microrregiao,primeiro_ano_estacao,num_estacoes
0,11001,2007,1
1,11002,2016,2
2,11003,2008,1
3,11006,2008,1
4,11007,2008,2


## 3. População e PIB

Agregar dados socioeconômicos por microrregião:


In [55]:
# PASSO 3: População
query_pop = f"""
SELECT
    ano,
    id_municipio,
    populacao
FROM
    `basedosdados.br_ibge_populacao.municipio`
WHERE
    ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
"""

print("👥 Baixando dados de população...")
df_pop_mun = bd.read_sql(query_pop, billing_project_id=PROJECT_ID)

# JOIN com mapeamento
df_pop_mapped = df_pop_mun.merge(
    df_municipios[['id_municipio', 'id_microrregiao', 'sigla_uf']], 
    on='id_municipio', 
    how='left'
)

# Agregar por microrregião
df_pop = df_pop_mapped.groupby(['ano', 'id_microrregiao', 'sigla_uf'])['populacao'].sum().reset_index()
df_pop.columns = ['ano', 'id_microrregiao', 'uf', 'populacao_total']

print(f"✅ {len(df_pop):,} registros agregados")

# PASSO 4: PIB
query_pib = f"""
SELECT
    ano,
    id_municipio,
    pib,
    va_agropecuaria
FROM
    `basedosdados.br_ibge_pib.municipio`
WHERE
    ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
"""

print("\n💰 Baixando dados de PIB...")
df_pib_mun = bd.read_sql(query_pib, billing_project_id=PROJECT_ID)

# JOIN com mapeamento
df_pib_mapped = df_pib_mun.merge(
    df_municipios[['id_municipio', 'id_microrregiao']], 
    on='id_municipio', 
    how='left'
)

# Agregar
df_pib = df_pib_mapped.groupby(['ano', 'id_microrregiao']).agg({
    'pib': 'sum',
    'va_agropecuaria': 'sum'
}).reset_index()

df_pib.columns = ['ano', 'id_microrregiao', 'pib_total', 'pib_agropecuario']

print(f"✅ {len(df_pib):,} registros agregados")


👥 Baixando dados de população...
Downloading: 100%|██████████|
✅ 11,160 registros agregados

💰 Baixando dados de PIB...
Downloading: 100%|██████████|
✅ 10,602 registros agregados


## 4. Consolidação do Dataset

Criar painel balanceado com todas as microrregiões e anos:


In [56]:
# PASSO 5: Criar painel
# Obter lista de todas as microrregiões
all_micros = sorted(df_municipios['id_microrregiao'].unique())
print(f"📊 Total de microrregiões: {len(all_micros)}")

# Criar painel balanceado
from itertools import product
painel = pd.DataFrame(
    list(product(all_micros, ANOS)), 
    columns=['id_microrregiao', 'ano']
)

# Adicionar informações básicas das microrregiões
micro_info = df_municipios.groupby('id_microrregiao')['sigla_uf'].first().reset_index()
painel = painel.merge(micro_info, on='id_microrregiao', how='left')

# Adicionar tratamento (estações)
painel = painel.merge(
    df_estacoes[['id_microrregiao', 'primeiro_ano_estacao']], 
    on='id_microrregiao', 
    how='left'
)

painel['primeiro_ano_tratamento'] = painel['primeiro_ano_estacao'].fillna(0).astype(int)
painel['tratado'] = (painel['primeiro_ano_tratamento'] != 0).astype(int)
painel['pos_tratamento'] = (
    (painel['ano'] >= painel['primeiro_ano_tratamento']) & 
    (painel['tratado'] == 1)
).astype(int)

# Adicionar população
painel = painel.merge(
    df_pop[['ano', 'id_microrregiao', 'populacao_total']], 
    on=['ano', 'id_microrregiao'], 
    how='left'
)

# Adicionar PIB
painel = painel.merge(
    df_pib, 
    on=['ano', 'id_microrregiao'], 
    how='left'
)

# PIB per capita
painel['pib_per_capita'] = (painel['pib_total'] / painel['populacao_total']).round(2)

print(f"\n✅ Painel criado: {len(painel):,} observações")
print(f"📊 Microrregiões tratadas: {painel[painel['tratado'] == 1]['id_microrregiao'].nunique()}")
print(f"📊 Microrregiões controle: {painel[painel['tratado'] == 0]['id_microrregiao'].nunique()}")

# Mostrar amostra
painel.head()


📊 Total de microrregiões: 558

✅ Painel criado: 11,718 observações
📊 Microrregiões tratadas: 394
📊 Microrregiões controle: 164


,id_microrregiao,ano,sigla_uf,primeiro_ano_estacao,primeiro_ano_tratamento,tratado,pos_tratamento,populacao_total,pib_total,pib_agropecuario,pib_per_capita
0,11001,2003,RO,2007,2007,1,0,448419,3173358000,170001000,7076.77
1,11001,2004,RO,2007,2007,1,0,497775,3885396000,175586000,7805.53
2,11001,2005,RO,2007,2007,1,0,485002,4385122000,195425000,9041.45
3,11001,2006,RO,2007,2007,1,0,497936,4723308000,192598000,9485.77
4,11001,2007,RO,2007,2007,1,1,474532,5169521000,230175000,10893.94


In [57]:
# PASSO 6: Dados REAIS de produção agrícola (PAM)

# Converter lista de produtos para formato SQL
produtos_sql = ', '.join([f"'{p}'" for p in PRODUTOS_AGRICOLAS])

query_pam = f"""
SELECT
    lav.id_municipio,
    lav.ano,
    SUM(lav.area_plantada) AS area_plantada,
    SUM(lav.area_colhida) AS area_colhida,
    SUM(lav.quantidade_produzida) AS quantidade_produzida,
    AVG(lav.rendimento_medio_producao) AS produtividade,
    SUM(lav.valor_producao) AS valor_producao
FROM
    `basedosdados.br_ibge_pam.lavoura_temporaria` AS lav
WHERE
    lav.ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
    AND lav.produto IN ({produtos_sql})
GROUP BY
    lav.id_municipio,
    lav.ano
"""

print(f"🌾 Baixando dados de produção de: {', '.join(PRODUTOS_AGRICOLAS)}...")
df_pam_mun = bd.read_sql(query_pam, billing_project_id=PROJECT_ID)
print(f"✅ {len(df_pam_mun):,} registros de produção municipal")

# JOIN com mapeamento para obter microrregião
df_pam_mapped = df_pam_mun.merge(
    df_municipios[['id_municipio', 'id_microrregiao']], 
    on='id_municipio', 
    how='left'
)

# Agregar por microrregião
df_pam = df_pam_mapped.groupby(['ano', 'id_microrregiao']).agg({
    'area_plantada': 'sum',
    'area_colhida': 'sum',
    'quantidade_produzida': 'sum',
    'valor_producao': 'sum',
    'produtividade': 'mean'  # Média ponderada seria ideal, mas por simplicidade
}).reset_index()

# Renomear colunas genericamente (sem sufixo _cana)
df_pam.columns = [
    'ano', 'id_microrregiao',
    'area_plantada', 'area_colhida', 
    'producao', 'valor_producao',
    'produtividade'
]

print(f"✅ {len(df_pam):,} registros agregados por microrregião")

# Adicionar ao painel
painel_final = painel.merge(
    df_pam, 
    on=['ano', 'id_microrregiao'], 
    how='left'
)

# Preencher zeros onde não há produção
cols_producao = ['area_plantada', 'area_colhida', 'producao', 
                 'valor_producao', 'produtividade']
for col in cols_producao:
    painel_final[col] = painel_final[col].fillna(0)

# NOVO: Calcular taxa de aproveitamento (área colhida / área plantada)
# Essa métrica mostra a eficiência da produção - quanto maior, melhor o aproveitamento
painel_final['taxa_aproveitamento'] = np.where(
    painel_final['area_plantada'] > 0,
    painel_final['area_colhida'] / painel_final['area_plantada'],
    0
)

# NOVO: Filtrar microrregiões com pelo menos 10 anos de produção
anos_com_producao = painel_final[painel_final['area_plantada'] > 0].groupby('id_microrregiao').size()
micros_validas = anos_com_producao[anos_com_producao >= 10].index
n_micros_antes = painel_final['id_microrregiao'].nunique()

# Aplicar filtro
painel_final = painel_final[painel_final['id_microrregiao'].isin(micros_validas)]
n_micros_depois = painel_final['id_microrregiao'].nunique()

# Remover coluna auxiliar
if 'primeiro_ano_estacao' in painel_final.columns:
    painel_final = painel_final.drop('primeiro_ano_estacao', axis=1)

print(f"\n✅ Dados de produção REAIS integrados!")
print(f"📊 Cobertura: {(painel_final['producao'] > 0).sum()} observações com produção")
print(f"📊 Microrregiões produtoras: {painel_final[painel_final['producao'] > 0]['id_microrregiao'].nunique()}")
print(f"\n🔍 Filtro de qualidade aplicado:")
print(f"   - Microrregiões antes do filtro: {n_micros_antes}")
print(f"   - Microrregiões após filtro (≥10 anos produção): {n_micros_depois}")
print(f"   - Microrregiões removidas: {n_micros_antes - n_micros_depois}")


🌾 Baixando dados de produção de: Cana-de-açúcar...
Downloading: 100%|██████████|
✅ 116,823 registros de produção municipal
✅ 11,718 registros agregados por microrregião

✅ Dados de produção REAIS integrados!
📊 Cobertura: 9932 observações com produção
📊 Microrregiões produtoras: 490

🔍 Filtro de qualidade aplicado:
   - Microrregiões antes do filtro: 558
   - Microrregiões após filtro (≥10 anos produção): 490
   - Microrregiões removidas: 68


In [58]:
# Verificar estatísticas básicas após filtro
print(f"\n📊 Estatísticas após filtro de qualidade:")
print(f"   - Taxa de aproveitamento média: {painel_final[painel_final['taxa_aproveitamento'] > 0]['taxa_aproveitamento'].mean():.1%}")
print(f"   - Produtividade média: {painel_final[painel_final['produtividade'] > 0]['produtividade'].mean():.1f} kg/ha")



📊 Estatísticas após filtro de qualidade:
   - Taxa de aproveitamento média: 99.1%
   - Produtividade média: 49240.3 kg/ha


## 7. Exportar Dataset Final


In [59]:
# PASSO 7: Exportar dataset
# Ordenar colunas
cols_order = [
    'ano', 'id_microrregiao', 'sigla_uf',
    'primeiro_ano_tratamento', 'tratado', 'pos_tratamento',
    'area_plantada', 'area_colhida', 'taxa_aproveitamento', 'producao', 'produtividade', 'valor_producao',
    'populacao_total', 'pib_total', 'pib_per_capita', 'pib_agropecuario'
]

# Selecionar colunas existentes
cols_final = [c for c in cols_order if c in painel_final.columns]
df_final = painel_final[cols_final].sort_values(['id_microrregiao', 'ano'])

# Criar nome do arquivo com produtos separados por hífen
produtos_filename = '-'.join(PRODUTOS_AGRICOLAS).replace(' ', '_').replace('(', '').replace(')', '')
output_file = f'data/microrregions_{produtos_filename}_{min(ANOS)}-{max(ANOS)}.csv'

# Criar diretório se não existir
os.makedirs('data', exist_ok=True)

# Salvar com nome descritivo
df_final.to_csv(output_file, index=False)

print(f"✅ Dataset exportado: {output_file}")
print(f"📏 Tamanho: {os.path.getsize(output_file) / 1024 / 1024:.1f} MB")
print(f"📊 Dimensões: {len(df_final):,} observações × {len(df_final.columns)} variáveis")

# Estatísticas finais
print(f"\n📊 Resumo do dataset filtrado:")
print(f"  - Microrregiões: {df_final['id_microrregiao'].nunique()} (com ≥10 anos de produção)")
print(f"  - Estados: {df_final['sigla_uf'].nunique()}")
print(f"  - Microrregiões tratadas: {df_final[df_final['tratado'] == 1]['id_microrregiao'].nunique()}")
print(f"  - Microrregiões controle: {df_final[df_final['tratado'] == 0]['id_microrregiao'].nunique()}")
print(f"  - Taxa de aproveitamento média: {df_final[df_final['taxa_aproveitamento'] > 0]['taxa_aproveitamento'].mean():.1%}")
print(f"  - Missing values população: {df_final['populacao_total'].isnull().sum()} ({df_final['populacao_total'].isnull().sum()/len(df_final)*100:.1f}%)")
print(f"  - Missing values PIB: {df_final['pib_total'].isnull().sum()} ({df_final['pib_total'].isnull().sum()/len(df_final)*100:.1f}%)")

# Salvar mapeamento município-microrregião
df_municipios.to_csv('output/mapeamento_municipio_microrregiao.csv', index=False)
print(f"\n✅ Mapeamento salvo: output/mapeamento_municipio_microrregiao.csv")

# Mostrar amostra final
df_final.head(10)


✅ Dataset exportado: data/microrregions_Cana-de-açúcar_2003-2023.csv
📏 Tamanho: 0.9 MB
📊 Dimensões: 10,290 observações × 16 variáveis

📊 Resumo do dataset filtrado:
  - Microrregiões: 490 (com ≥10 anos de produção)
  - Estados: 27
  - Microrregiões tratadas: 351
  - Microrregiões controle: 139
  - Taxa de aproveitamento média: 99.1%
  - Missing values população: 490 (4.8%)
  - Missing values PIB: 980 (9.5%)

✅ Mapeamento salvo: output/mapeamento_municipio_microrregiao.csv


,ano,id_microrregiao,sigla_uf,primeiro_ano_tratamento,tratado,pos_tratamento,area_plantada,area_colhida,taxa_aproveitamento,producao,produtividade,valor_producao,populacao_total,pib_total,pib_per_capita,pib_agropecuario
0,2003,11001,RO,2007,1,0,82,82,1.0,5695.0,66000.000000,226.0,448419,3173358000,7076.77,170001000
1,2004,11001,RO,2007,1,0,99,99,1.0,6817.0,68142.857143,291.0,497775,3885396000,7805.53,175586000
2,2005,11001,RO,2007,1,0,92,92,1.0,6111.0,65957.142857,271.0,485002,4385122000,9041.45,195425000
3,2006,11001,RO,2007,1,0,117,117,1.0,7645.0,64558.571429,440.0,497936,4723308000,9485.77,192598000
4,2007,11001,RO,2007,1,1,75,75,1.0,4657.0,61917.428571,318.0,474532,5169521000,10893.94,230175000
5,2008,11001,RO,2007,1,1,78,78,1.0,4576.0,57257.142857,327.0,486884,6118513000,12566.68,340318000
6,2009,11001,RO,2007,1,1,104,104,1.0,5827.0,56335.714286,233.0,493445,7786946000,15780.78,367499000
7,2010,11001,RO,2007,1,1,73,73,1.0,4088.0,56000.000000,163.0,540320,10298189000,19059.43,426759000
8,2011,11001,RO,2007,1,1,47,47,1.0,2587.0,55255.500000,760.0,550089,12509934000,22741.65,508095000
9,2012,11001,RO,2007,1,1,0,0,0.0,0.0,0.000000,0.0,559543,13425222000,23993.19,633448000
